In [10]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import ast # Asegurarnos que ast esté importado

print("--- [FIX] Re-creando las columnas de 'parsing' ---")

# --- 1. Definir las funciones de parsing (NECESARIO) ---

def safe_literal_eval(s):
    try:
        return ast.literal_eval(s)
    except (ValueError, SyntaxError, TypeError):
        return []

def get_names(data_list_str):
    data_list = safe_literal_eval(data_list_str)
    if isinstance(data_list, list):
        names = [item['name'].lower().replace(" ", "") for item in data_list if 'name' in item]
        return names
    return []

def get_top_cast(cast_list_str, top_n=3):
    cast_list = safe_literal_eval(cast_list_str)
    if isinstance(cast_list, list):
        names = [item['name'].lower().replace(" ", "") for item in cast_list[:top_n] if 'name' in item]
        return names
    return []

def get_director(crew_list_str):
    crew_list = safe_literal_eval(crew_list_str)
    if isinstance(crew_list, list):
        for member in crew_list:
            if member.get('job') == 'Director':
                return member.get('name', '').lower().replace(" ", "")
    return ""

print("Funciones de parsing definidas.")

# --- 2. Limpiar NaNs (NECESARIO) ---
# (Estos son los strings que parecen listas)
df['genres'] = df['genres'].fillna('[]')
df['keywords'] = df['keywords'].fillna('[]')
df['cast'] = df['cast'].fillna('[]')
df['crew'] = df['crew'].fillna('[]')

# --- 3. Aplicar las funciones para crear las columnas (EL FIX) ---
print("Aplicando funciones para crear 'genres_list', 'keywords_list', y 'cast_list'...")
df['genres_list'] = df['genres'].apply(get_names)
df['keywords_list'] = df['keywords'].apply(get_names)
df['cast_list'] = df['cast'].apply(get_top_cast)
# (director ya debería existir, pero lo re-creamos por si acaso)
df['director'] = df['crew'].apply(get_director)

print("\n¡FIX APLICADO! La columna 'genres_list' ahora existe.")
print("--- [INICIO] ANÁLISIS EXPLORATORIO DE DATOS (EDA) ---")

# Configuración de estilo profesional para los gráficos
sns.set(style="whitegrid")

# --- Gráfico 1: Proporción de Películas vs. Series ---
# (Este gráfico depende de si arreglaste la columna 'type' como vimos)
try:
    print("Generando Gráfico 1: Tipo de Contenido...")
    # Filtrar solo 'Movie' y 'TV Show' válidos
    valid_types = ['Movie', 'TV Show']
    df_filtered_types = df[df['type'].isin(valid_types)]

    type_counts = df_filtered_types['type'].value_counts()

    plt.figure(figsize=(10, 6))
    sns.countplot(x='type', data=df_filtered_types, order=type_counts.index, palette="viridis")
    plt.title('Proporción de Películas vs. Series de TV', fontsize=16)
    plt.xlabel('Tipo de Contenido', fontsize=12)
    plt.ylabel('Cantidad (Conteo)', fontsize=12)

    # Guardar el gráfico en Colab
    plt.savefig('grafico_1_tipo_contenido.png')
    plt.close() # Cierra el gráfico para ahorrar memoria
    print("Gráfico 1 guardado como 'grafico_1_tipo_contenido.png'")

except KeyError:
    print("ADVERTENCIA: Columna 'type' no encontrada. Omitiendo Gráfico 1.")
except Exception as e:
    print(f"Error al generar Gráfico 1: {e}")


# --- Gráfico 2: Distribución de Años de Estreno ---
print("\nGenerando Gráfico 2: Años de Estreno...")
try:
    # 1. Limpieza de 'release_date'
    release_dates = pd.to_datetime(df['release_date'], errors='coerce')

    # 2. Extraer el año y eliminar valores nulos
    release_years = release_dates.dt.year.dropna()

    # 3. Filtrar años irreales (ej. solo nos interesa desde 1900 hasta el año actual)
    release_years = release_years[(release_years > 1900) & (release_years <= 2025)]

    # 4. Crear el histograma
    plt.figure(figsize=(14, 7))
    sns.histplot(release_years, bins=60, kde=False, color="blue")
    plt.title('Distribución de Años de Estreno del Contenido', fontsize=16)
    plt.xlabel('Año de Estreno', fontsize=12)
    plt.ylabel('Cantidad de Títulos', fontsize=12)

    # 5. Guardar el gráfico
    plt.savefig('grafico_2_anios_estreno.png')
    plt.close()
    print("Gráfico 2 guardado como 'grafico_2_anios_estreno.png'")

except Exception as e:
    print(f"Error al generar Gráfico 2: {e}")


# --- Gráfico 3: Top 15 Géneros (Análisis de "Larga Cola") ---
print("\nGenerando Gráfico 3: Top 15 Géneros...")
try:
    # 1. Usar .explode() para separar las listas de 'genres_list'
    # (Esta columna debió crearse en la fase de Preparación)
    genres_exploded = df.explode('genres_list')

    # 2. Contar los géneros (y excluir strings vacíos si los hay)
    top_genres = genres_exploded[genres_exploded['genres_list'] != '']['genres_list'].value_counts().head(15)

    # 3. Crear el gráfico de barras
    plt.figure(figsize=(14, 8))
    sns.barplot(x=top_genres.values, y=top_genres.index, palette='mako')
    plt.title('Top 15 Géneros Más Comunes en el Catálogo', fontsize=16)
    plt.xlabel('Cantidad (Conteo)', fontsize=12)
    plt.ylabel('Género', fontsize=12)

    # 4. Guardar el gráfico
    plt.savefig('grafico_3_top_generos.png')
    plt.close()
    print("Gráfico 3 guardado como 'grafico_3_top_generos.png'")

except KeyError:
    print("ERROR: Columna 'genres_list' no encontrada. ¿Ejecutaste el paso de 'parsing'?")
except Exception as e:
    print(f"Error al generar Gráfico 3: {e}")


# --- Gráfico 4: Distribución de Calificaciones (Vote Average) ---
print("\nGenerando Gráfico 4: Distribución de Calificaciones...")
try:
    # 1. Asegurarnos que 'vote_average' es numérico
    df['vote_average'] = pd.to_numeric(df['vote_average'], errors='coerce')

    # 2. Filtrar películas sin votos (vote_average == 0) para no sesgar el gráfico
    votos_reales = df[df['vote_average'] > 0]['vote_average']

    # 3. Crear el histograma con curva de densidad (KDE)
    plt.figure(figsize=(12, 7))
    sns.histplot(votos_reales, bins=40, kde=True, color="purple")
    plt.title('Distribución de Calificaciones Promedio (Vote Average)', fontsize=16)
    plt.xlabel('Calificación Promedio (de 0 a 10)', fontsize=12)
    plt.ylabel('Densidad', fontsize=12)

    # 4. Añadir línea de promedio
    mean_val = votos_reales.mean()
    plt.axvline(mean_val, color='red', linestyle='--', label=f'Promedio: {mean_val:.2f}')
    plt.legend()

    # 5. Guardar el gráfico
    plt.savefig('grafico_4_distribucion_votos.png')
    plt.close()
    print("Gráfico 4 guardado como 'grafico_4_distribucion_votos.png'")

except Exception as e:
    print(f"Error al generar Gráfico 4: {e}")


# --- Gráfico 5: Matriz de Correlación ---
print("\nGenerando Gráfico 5: Matriz de Correlación...")
try:
    # 1. Convertir TODAS las columnas numéricas clave (¡MUY IMPORTANTE!)
    # 'budget' y 'revenue' están como strings en el CSV.
    df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
    df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
    df['popularity'] = pd.to_numeric(df['popularity'], errors='coerce')
    df['runtime'] = pd.to_numeric(df['runtime'], errors='coerce')
    df['vote_count'] = pd.to_numeric(df['vote_count'], errors='coerce')
    # 'vote_average' ya lo convertimos en el paso anterior

    # 2. Crear un subset solo con películas que tengan datos realistas
    df_numeric = df[
        (df['budget'] > 1000) &      # Presupuesto > 1000
        (df['revenue'] > 1000) &     # Ganancias > 1000
        (df['vote_count'] > 100)   # Solo películas con > 100 votos
    ][['budget', 'revenue', 'popularity', 'runtime', 'vote_average', 'vote_count']]

    # 3. Calcular la matriz de correlación
    corr_matrix = df_numeric.corr()

    # 4. Crear el mapa de calor (Heatmap)
    plt.figure(figsize=(12, 8))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', linewidths=.5)
    plt.title('Matriz de Correlación de Variables Numéricas', fontsize=16)
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)

    # 5. Guardar el gráfico
    plt.savefig('grafico_5_correlacion.png')
    plt.close()
    print("Gráfico 5 guardado como 'grafico_5_correlacion.png'")

except Exception as e:
    print(f"Error al generar Gráfico 5: {e}")

print("\n--- [FIN] ANÁLISIS EXPLORATORIO DE DATOS ---")

--- [FIX] Re-creando las columnas de 'parsing' ---
Funciones de parsing definidas.
Aplicando funciones para crear 'genres_list', 'keywords_list', y 'cast_list'...

¡FIX APLICADO! La columna 'genres_list' ahora existe.
--- [INICIO] ANÁLISIS EXPLORATORIO DE DATOS (EDA) ---
Generando Gráfico 1: Tipo de Contenido...
ADVERTENCIA: Columna 'type' no encontrada. Omitiendo Gráfico 1.

Generando Gráfico 2: Años de Estreno...
Gráfico 2 guardado como 'grafico_2_anios_estreno.png'

Generando Gráfico 3: Top 15 Géneros...


/tmp/ipython-input-690187926.py:128: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=top_genres.values, y=top_genres.index, palette='mako')


Gráfico 3 guardado como 'grafico_3_top_generos.png'

Generando Gráfico 4: Distribución de Calificaciones...
Gráfico 4 guardado como 'grafico_4_distribucion_votos.png'

Generando Gráfico 5: Matriz de Correlación...
Gráfico 5 guardado como 'grafico_5_correlacion.png'

--- [FIN] ANÁLISIS EXPLORATORIO DE DATOS ---
